In [10]:
%matplotlib inline
import torch
import torchvision
from IPython import display
from matplotlib import pyplot as plt
import numpy as np
import random

In [11]:
def use_svg_display():
    # 用矢量图显示
    display.set_matplotlib_formats('svg')

def set_figsize(figsize=(3.5, 2.5)):
    use_svg_display()
    # 设置图的尺寸
    plt.rcParams['figure.figsize'] = figsize

In [12]:
!ls ./data/iris

iris_test.csv     iris_training.csv


In [13]:
class IrisDataSet(torch.utils.data.Dataset):
    def __init__(self, csv_path, sep=','):
        with open(csv_path, 'r') as f:
            lines = f.readlines()[1:]
            datas = [line.strip().split(sep) for line in lines]
            datas_np = np.array(datas, dtype=np.float32)
            self.featurs = torch.tensor(datas_np[:,:4], dtype=torch.float32)
            self.labels = torch.tensor(datas_np[:,-1], dtype=torch.float32)
    
    def __len__(self):
        return len(self.featurs)
    
    def __getitem__(self, index):
        return self.featurs[index], self.labels[index]
    

In [14]:
batch_size = 32
train_csv_path = './data/iris/iris_training.csv'
train_data_set = IrisDataSet(train_csv_path)
train_data_iter = torch.utils.data.DataLoader(train_data_set, batch_size=batch_size, shuffle=True)

test_csv_path = './data/iris/iris_test.csv'
test_data_set = IrisDataSet(test_csv_path)
test_data_iter = torch.utils.data.DataLoader(test_data_set, batch_size=batch_size)

print(len(train_data_set), len(test_data_set))

120 30


In [15]:
w = torch.randn((4,3), dtype=torch.float32)
b = torch.zeros((1,3), dtype=torch.float32)

w.requires_grad_(True)
b.requires_grad_(True)

w, b

(tensor([[ 2.2572,  0.2277, -0.0742],
         [ 0.5927, -0.8022, -0.1140],
         [-0.0041,  0.6776,  0.4867],
         [-0.1093,  1.0666,  0.4089]], requires_grad=True),
 tensor([[0., 0., 0.]], requires_grad=True))

In [37]:
def softmax(x):
    x = x - x.max(dim=1, keepdim=True).values
    return x.exp() / x.exp().sum(dim=1, keepdim=True)

In [42]:
def cross_entropy_loss(y, y_hat):
    """
    y=[0,1,2]
    y_hat = [
        [0.7,0.2,0.1],
        [0.2,0.5,0.3],
        [0.1,0.1,0.8],
    ]
    """
    y_hat = softmax(y_hat)
    return -torch.log(
        y_hat.gather(dim=1, index=y.type(torch.long).view(-1,1))+1e-5
    ).sum()

def sgd(params, lr, bs):
    for param in params:
        param.data -= param.grad / bs * lr
        
def net(x):
    return torch.mm(x, w) + b
        
def accuracy(y_hat, y):
    return (y_hat.argmax(dim=1) == y).float().mean().item()

def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0.0, 0
    for X, y in data_iter:
        acc_sum += (net(X).argmax(dim=1) == y).float().sum().item()
        n += y.shape[0]
    return acc_sum / n

In [43]:
lr = 0.1
num_epochs = 20
params = [w, b]
loss = cross_entropy_loss
for epoch in range(1, num_epochs+1):
    train_l_sum, train_acc_sum, n = 0.0, 0.0, 0
    for x, y in train_data_iter:
        for param in params:
            if param.grad is not None:
                param.grad.data.zero_()
        y_hat = net(x)
        l = cross_entropy(y, y_hat)
        l.backward()
        sgd(params, lr, batch_size)
        
        train_l_sum += l.item()
        train_acc_sum += (y_hat.argmax(dim=1)==y).float().sum().item()
        n += y.shape[0]
    print('epoch %d loss %.4f train_acc %.4f test_acc %.4f' %
          (epoch, train_l_sum/n, train_acc_sum/n, evaluate_accuracy(test_data_iter, net)))
        

epoch 1 loss 0.2360 train_acc 0.9333 test_acc 0.8667
epoch 2 loss 0.1774 train_acc 0.9250 test_acc 0.9667
epoch 3 loss 0.1892 train_acc 0.9583 test_acc 0.9333
epoch 4 loss 0.1751 train_acc 0.9500 test_acc 0.9667
epoch 5 loss 0.2021 train_acc 0.9333 test_acc 0.9667
epoch 6 loss 0.1870 train_acc 0.9500 test_acc 0.9667
epoch 7 loss 0.1890 train_acc 0.9500 test_acc 0.9333
epoch 8 loss 0.1728 train_acc 0.9750 test_acc 0.9667
epoch 9 loss 0.2075 train_acc 0.9417 test_acc 1.0000
epoch 10 loss 0.1977 train_acc 0.9250 test_acc 0.9667
epoch 11 loss 0.1840 train_acc 0.9500 test_acc 0.9667
epoch 12 loss 0.1886 train_acc 0.9583 test_acc 0.9333
epoch 13 loss 0.1722 train_acc 0.9750 test_acc 1.0000
epoch 14 loss 0.1749 train_acc 0.9583 test_acc 0.9333
epoch 15 loss 0.1765 train_acc 0.9667 test_acc 1.0000
epoch 16 loss 0.1686 train_acc 0.9750 test_acc 0.9333
epoch 17 loss 0.1724 train_acc 0.9750 test_acc 0.9333
epoch 18 loss 0.1698 train_acc 0.9750 test_acc 0.9667
epoch 19 loss 0.1663 train_acc 0.9750